In [ ]:
from manim import *
import random

from manim.utils.color.AS2700 import Y14_GOLDEN_YELLOW

class PRO(Scene):
    def construct(self):
        t1 = Text("Bagging", font="Microsoft YaHei", font_size=110, color=GOLD_A, weight="BOLD")
        t1.move_to(UP*4).set_opacity(0)

        self.play(t1.animate.shift(DOWN*4).set_opacity(1))
        self.wait(0.5)
        self.play(t1.animate.shift(DOWN*5).set_opacity(0))

        # 固定随机种子
        random.seed(42)

        # 表格总体尺寸
        total_width = 12.0
        total_height = 0.6
        cols = 20

        # 外框矩形
        outer = Rectangle(
            width=total_width,
            height=total_height,
            stroke_color=WHITE,
            stroke_width=2,
            fill_opacity=0,
        )
        outer.to_edge(UP, buff=0.5)

        # 每个小方格
        col_w = total_width / cols
        squares = VGroup()
        left_x = outer.get_left()[0]
        bottom_y = outer.get_bottom()[1]
        for i in range(cols):
            sq = Rectangle(
                width=col_w,
                height=total_height,
                stroke_color=WHITE,
                stroke_width=1,
                fill_opacity=0
            )
            sq.move_to(np.array([left_x + (i + 0.5) * col_w, bottom_y + total_height / 2, 0]))
            squares.add(sq)

        # 动画：先画外框和格子
        self.play(Create(outer))
        self.play(LaggedStart(*[Create(s) for s in squares], lag_ratio=0.02))
        self.wait(0.5)

        # ---------- 第一次：随机红色 ----------
        chosen_indices_red = random.sample(range(cols), 10)
        chosen_squares_red = VGroup(*[squares[i] for i in chosen_indices_red])
        self.play(*[sq.animate.set_fill(RED, opacity=1) for sq in chosen_squares_red])
        self.wait(0.5)

        new_row_red = VGroup(*[sq.copy() for sq in chosen_squares_red])
        new_row_red.arrange(RIGHT, buff=0)
        new_row_red.next_to(outer, DOWN, buff=1).scale(0.5).shift(LEFT*4.4)
        
        self.play(
            TransformFromCopy(chosen_squares_red, new_row_red),
            *[sq.animate.set_fill(opacity=0) for sq in chosen_squares_red]
        )
        self.wait(0.5)

        # ---------- 第二次：随机蓝色 ----------
        chosen_indices_blue = random.sample(range(cols), 10)
        chosen_squares_blue = VGroup(*[squares[i] for i in chosen_indices_blue])
        self.play(*[sq.animate.set_fill(BLUE, opacity=1) for sq in chosen_squares_blue])
        self.wait(0.5)

        new_row_blue = VGroup(*[sq.copy() for sq in chosen_squares_blue])
        new_row_blue.arrange(RIGHT, buff=0)
        new_row_blue.next_to(new_row_red, RIGHT, buff=0.9).scale(0.5).shift(LEFT*1)
        
        self.play(
            TransformFromCopy(chosen_squares_blue, new_row_blue),
            *[sq.animate.set_fill(opacity=0) for sq in chosen_squares_blue]
        )
        self.wait(1)
        # ---------- 第三次：绿色 ----------
        chosen_indices_green = random.sample(range(cols), 10)
        chosen_squares_green = VGroup(*[squares[i] for i in chosen_indices_green])
        self.play(*[sq.animate.set_fill(GREEN, opacity=1) for sq in chosen_squares_green])
        self.wait(0.5)

        new_row_green = VGroup(*[sq.copy() for sq in chosen_squares_green])
        new_row_green.arrange(RIGHT, buff=0)
        new_row_green.next_to(new_row_blue, RIGHT, buff=0.9).scale(0.5).shift(LEFT*1)
        
        self.play(
            TransformFromCopy(chosen_squares_green, new_row_green),
            *[sq.animate.set_fill(opacity=0) for sq in chosen_squares_green]
        )
        self.wait(1)
        # ---------- 三个表格整体下移 ----------
        all_tables = VGroup(new_row_blue, new_row_green, new_row_red)
        t2 = Text("n", font="Microsoft YaHei", font_size=110, color=GOLD_A, weight="BOLD")
        t2.next_to(outer, DOWN).shift(DOWN*0.5)
        self.play(
            all_tables.animate.shift(DOWN*1),
            Write(t2)
            )

        self.wait(2)

        t3 = Text("数据重采样", font="Microsoft YaHei", font_size=80, color=GOLD_C, weight="BOLD")
        t3.next_to(outer, DOWN).shift(DOWN*4.5).set_opacity(0)
        t4 = Text("制造差异", font="Microsoft YaHei", font_size=60, color=GOLD_A, weight="BOLD")
        t4.next_to(outer, DOWN).shift(DOWN*4.5).set_opacity(0)       
        self.play(t3.animate.shift(UP*0.5).set_opacity(1))
        self.wait(1)
        self.play(
            t3.animate.shift(UP*1).scale(0.85),
            t4.animate.shift(UP*0.3).set_opacity(1)
        )
        self.wait(1)
        self.play(
            t2.animate.set_opacity(0).shift(UP*0.5),
            t3.animate.set_opacity(0).shift(UP*0.5),
            t4.animate.set_opacity(0).shift(UP*0.5),
            all_tables.animate.shift(UP*1)
        )
        self.wait(1)
        # ---------- 三个小表格下方画箭头和矩形 ----------
        colors = [RED, BLUE, GREEN]
        arrow_group = VGroup()
        learner_group = VGroup()
        for idx, (table, color) in enumerate(zip([new_row_red, new_row_blue, new_row_green], colors), start=1):
            # 表格指向矩形的箭头
            arrow = Arrow(
                start=table.get_bottom() + DOWN*0.1,
                end=table.get_bottom() + DOWN*0.5,
                buff=0,
                color=color
            )
            arrow_group.add(arrow)
            
            # 矩形
            rect = Rectangle(width=2, height=0.8, stroke_color=color, stroke_width=2, fill_opacity=0.2, fill_color=color)
            rect.next_to(arrow, DOWN, buff=0.2)
            learner_group.add(rect)
            
            # 矩形内文字，带编号，颜色一致
            text = Text(f"基学习器{idx}", font="Microsoft YaHei", font_size=30, color=color)
            text.move_to(rect.get_center())
            learner_group.add(text)

        self.play(
            *[Create(a) for a in arrow_group],
            *[Create(l) for l in learner_group]
        )
        self.wait(1)

        # ---------- 每个矩形下面画箭头到各自椭圆 ----------
        ellipse_group = VGroup()
        ellipse_arrow_group = VGroup()
        for idx, (rect, color) in enumerate(zip(learner_group[::2], colors), start=1):  # 偶数索引是矩形
            # 箭头
            arrow = Arrow(
                start=rect.get_bottom() + DOWN*0.1,
                end=rect.get_bottom() + DOWN*0.5,
                buff=0,
                color=color
            )
            ellipse_arrow_group.add(arrow)
            # 椭圆
            ellipse = Ellipse(width=2, height=0.9, stroke_color=color, fill_opacity=0.2, fill_color=color)
            ellipse.next_to(arrow, DOWN, buff=0.1)
            ellipse_group.add(ellipse)
            # 椭圆内文字，带编号，颜色一致
            text = Text(f"结果{idx}", font="Microsoft YaHei", font_size=30, color=color)
            text.move_to(ellipse.get_center())
            ellipse_group.add(text)

        self.play(
            *[Create(a) for a in ellipse_arrow_group],
            *[Create(e) for e in ellipse_group]
        )
        self.wait(5)
        # ---------- 三条箭头从结果椭圆指向中间"最终结果" ----------
        final_ellipse = Ellipse(width=3, height=1.5, stroke_color=WHITE, fill_opacity=0.2, fill_color=WHITE)
        final_ellipse.next_to(ellipse_group, DOWN, buff=0.7)  # 在三条结果椭圆下方

        final_text = Text("最终结果", font="Microsoft YaHei", font_size=40, color=WHITE)
        final_text.move_to(final_ellipse.get_center())

        final_arrows = VGroup()
        for ell in ellipse_group[::2]:  # 偶数索引是结果椭圆
            arrow = Arrow(
                start=ell.get_bottom() + DOWN*0.05,
                end=final_ellipse.get_top() + UP*0.05,
                buff=0,
                color=WHITE
            )
            final_arrows.add(arrow)

        # ---------- 最终结果左右文字 ----------
        left_text = Text("回归 → 平均", font="Microsoft YaHei", font_size=45, color=GOLD_B)
        left_text.next_to(final_ellipse, LEFT, buff=1)
        right_text = Text("分类 → 投票", font="Microsoft YaHei", font_size=45, color=GOLD_B)
        right_text.next_to(final_ellipse, RIGHT, buff=1)

        self.play(
            *[Create(a) for a in final_arrows],
            Create(final_ellipse),
            Write(final_text),
            Write(left_text),
            Write(right_text)
        )
        self.wait(4)

        gray_screen = Rectangle(
            width=self.camera.frame_width,
            height=self.camera.frame_height,
            fill_color=DARK_GRAY,
            fill_opacity=0.95,
            stroke_width=0,
            z_index=0
        )
        t5 = Text("Bagging", font="Microsoft YaHei", font_size=110, color=GOLD_A, weight="BOLD", z_index=2)
        t5.move_to(UP*4).set_opacity(0)
        # ---------- 最后总结文字 ----------
        summary_texts = [
            "1. 有放回抽样 - 数据重采样",
            "2. 得到多个不同的子数据集",
            "3. 每个子数据集上训练一个基学习器",
            "4. 平均 / 投票得最终输出结果"
        ]

        summary_group = VGroup()
        for i, line in enumerate(summary_texts):
            txt = Text(line, font="Microsoft YaHei", font_size=40, color=WHITE, z_index=1)
            txt.next_to(t5, DOWN, buff=0.8 + i*0.8).shift(DOWN*2)  # 每行间隔0.8
            summary_group.add(txt)
        # ---------- 补充说明文字 ----------
        t6 = Text("随机抽取 + 平均/投票", font="Microsoft YaHei", font_size=65, color=GOLD_B, weight="BOLD")
        t6.move_to(ORIGIN).shift(DOWN*0.5)
        t7 = Text("随机误差↓", font="Microsoft YaHei", font_size=65, color=GOLD_C, weight="BOLD")
        t7.move_to(ORIGIN).shift(DOWN*0.5)
        t8 = Text("方差↓", font="Microsoft YaHei", font_size=65, color=GOLD_D, weight="BOLD")
        t8.move_to(ORIGIN).shift(DOWN*0.5)
        t9 = Text("过拟合↓", font="Microsoft YaHei", font_size=65, color=GOLD_E, weight="BOLD")
        t9.move_to(ORIGIN).shift(DOWN*0.5)
        t10 = Text("欠拟合", font="Microsoft YaHei", font_size=65, color=RED_C, weight="BOLD")
        t10.move_to(ORIGIN).shift(DOWN*0.5+RIGHT*2.5)
        line = Line(
            start=t10.get_left() + UP*0.8,
            end=t10.get_right() + UP*0.8,
            color=RED_C,
            stroke_width=6 
        )

        self.play(
            FadeIn(gray_screen),
            t5.animate.shift(DOWN*2).set_opacity(1),
            *[FadeIn(txt) for txt in summary_group]
        )
        self.wait(5)
        self.play(
            *[txt.animate.shift(DOWN*5) for txt in summary_group],
            FadeIn(t6)
        )
        self.wait(2)
        self.play(Transform(t6, t7))
        self.wait(2)
        self.play(Transform(t6, t8))
        self.wait(2)
        self.play(Transform(t6, t9))
        self.wait(5)
        self.play(
            t6.animate.shift(LEFT*2.5).set_color(GREEN_C),
            FadeIn(t10, line)
        )
        self.wait(5)